In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
Xtest = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
Xtrain = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')

In [ ]:
Xtrain.head()

In [ ]:
ytrain = Xtrain['label']

Xtrain = Xtrain[Xtrain.columns[1:]]
Xtrain = Xtrain / 255
Xtest = Xtest / 255

In [ ]:
Samples,Height,Width = Xtrain.shape[0],28,28

In [ ]:
Xtrain = Xtrain.values.reshape(Samples,Width,Height,1).astype('float32')
Xtrain.shape

In [ ]:
Xtest.shape

In [ ]:
Xtest = Xtest.values.reshape(Xtest.shape[0],Height,Width,1).astype('float32')
Xtest.shape

In [ ]:
from functools import partial

In [ ]:
from tensorflow import keras

In [ ]:
Conv2D = partial(keras.layers.Conv2D,kernel_size=3,activation='relu',padding='SAME')

In [ ]:
DefaultConv2D = partial(keras.layers.Conv2D,kernel_size=3, activation='relu', padding="SAME")

model = keras.models.Sequential([
    DefaultConv2D(filters=64, kernel_size=7, input_shape=[28, 28, 1]),
    keras.layers.MaxPooling2D(pool_size=2),
    DefaultConv2D(filters=128),
    DefaultConv2D(filters=128),
    keras.layers.MaxPooling2D(pool_size=2),
    DefaultConv2D(filters=256),
    DefaultConv2D(filters=256),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(units=128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=64, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(units=10, activation='softmax'),
])

In [ ]:
early_stopping = keras.callbacks.EarlyStopping(patience=10)

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.fit(Xtrain,ytrain,shuffle=True,epochs=30,callbacks=[early_stopping])

In [ ]:
predict = model.predict(Xtest)

In [ ]:
classes = np.argmax(predict,axis=1)
classes[0]

In [ ]:
sample_submission = pd.read_csv('/kaggle/input/digit-recognizer/sample_submission.csv')
sample_submission.head()

In [ ]:
output_file = open('digits1.csv','w')

In [ ]:
classes.shape

In [ ]:
output_file.write('ImageId,Label\n')

In [ ]:
for i,value in enumerate(classes):
    output_file.write('{},{}\n'.format(i,int(value)))
    
output_file.close()